In [1]:
from data_parser import VelParser
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import shutil
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESSED_PATH = 'compressed'


class Compressor:
    def __init__(self):
        if not os.path.exists(COMPRESSED_PATH):
            os.mkdir(COMPRESSED_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
        self.parser = VelParser()
    
    def total_compress(self, data: np.ndarray) -> List[np.ndarray | np.float64]:
        wavelet_compressed = self.wavelet_compressor.wavelet_transform(data)[0]
        quantized, q_step, min_val = self.quantizator.uniform_quantization(wavelet_compressed)
        entropy_compressed = self.entropy_compressor.compress(quantized).hex()
        return [entropy_compressed, q_step, min_val]

    def save_mesh(self, mesh_data: Dict[str, np.ndarray]) -> None:
        # faces, boundary, neighbour, owner, pts = mesh_data.values()
        with open(COMPRESSED_PATH + '/' + 'mesh', mode='w+') as f:
            print(list(mesh_data.keys())[:-1])
            for (key, value) in list(mesh_data.items())[:-1]:
                compressed_ndarray, q_step, min_val = self.total_compress(value)
                f.write(f'{q_step} {min_val}\n')
                f.write(str(compressed_ndarray) + '\n')

    def compress_and_save(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        pprint(vels)
        mesh_data = self.parser.get_mesh_data(
            path_to_vel=vels[0],
        )
        print(mesh_data.keys())
        self.save_mesh(mesh_data=mesh_data)

        for vel in vels:
            vellast = vel.split('/')[-1]
            data = self.parser.get_vel_data(
                vel=vellast,
                base_path=base_path,
                model=model,
                dimpath=dim,
            )
            with open(COMPRESSED_PATH + '/' + vellast, mode='w+') as f:
                times = list(data.keys())
                f.write(f'{";".join(times)}\n')
                for (_, value) in list(data.items())[1:]:
                    p, t, u, rho = value
                    compressed_p, p_q_step, p_min_val = self.total_compress(p)
                    compressed_t, t_q_step, t_min_val = self.total_compress(t)
                    compressed_u, u_q_step, u_min_val = self.total_compress(u)
                    compressed_rho, rho_q_step, rho_min_val = self.total_compress(rho)
                    f.write(f'{p_q_step} {p_min_val}\n')
                    f.write(f'{str(compressed_p)}' + '\n')
                    f.write(f'{t_q_step} {t_min_val}\n')
                    f.write(f'{str(compressed_t)}' + '\n')
                    f.write(f'{u_q_step} {u_min_val}\n')
                    f.write(f'{str(compressed_u)}' + '\n')
                    f.write(f'{rho_q_step} {rho_min_val}\n')
                    f.write(f'{str(compressed_rho)}' + '\n')
    
    def copy_not_compressable(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        zero_vel = vels[0]
        thermophysical_path = f'{zero_vel}/constant/thermophysicalProperties'
        turbulenceProperties = f'{zero_vel}/constant/turbulenceProperties'
        system_path = f'{zero_vel}/system'
        destination_path = f'{COMPRESSED_PATH}/not_compressable'
        shutil.copytree(system_path, destination_path)
        shutil.copy(thermophysical_path, destination_path)
        shutil.copy(turbulenceProperties, destination_path)

SyntaxError: invalid syntax (wavelet_compressor.py, line 35)

In [3]:
compressor = Compressor()
data = compressor.compress_and_save(
    model='data_wage',
    base_path='data',
    dim='low_dim',
)

['data/data_wage/low_dim/vel3.0',
 'data/data_wage/low_dim/vel3.04040404040404',
 'data/data_wage/low_dim/vel3.080808080808081',
 'data/data_wage/low_dim/vel3.121212121212121',
 'data/data_wage/low_dim/vel3.1616161616161618',
 'data/data_wage/low_dim/vel3.202020202020202',
 'data/data_wage/low_dim/vel3.242424242424242',
 'data/data_wage/low_dim/vel3.282828282828283',
 'data/data_wage/low_dim/vel3.323232323232323',
 'data/data_wage/low_dim/vel3.3636363636363638',
 'data/data_wage/low_dim/vel3.404040404040404',
 'data/data_wage/low_dim/vel3.4444444444444446',
 'data/data_wage/low_dim/vel3.484848484848485',
 'data/data_wage/low_dim/vel3.525252525252525',
 'data/data_wage/low_dim/vel3.5656565656565657',
 'data/data_wage/low_dim/vel3.606060606060606',
 'data/data_wage/low_dim/vel3.6464646464646466',
 'data/data_wage/low_dim/vel3.686868686868687',
 'data/data_wage/low_dim/vel3.7272727272727275',
 'data/data_wage/low_dim/vel3.7676767676767677',
 'data/data_wage/low_dim/vel3.808080808080808',


/Users/stanislavtomulets/Desktop/aviahack/venv/lib/python3.11/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [4]:
vels = list(map(str, sorted(compressor.parser.get_vels(
    model='data_wage',
    base_path='data',
    dimpath='low_dim',
), key=lambda x: str(x))))
mesh_data = compressor.parser.get_mesh_data(
    path_to_vel=vels[0],
)

In [6]:
compressor.copy_not_compressable(
    model='data_wage',
    base_path='data',
    dim='low_dim'
)

In [10]:
def doall(
    model: str,
    base_path: str,
    dim: str,
):
    compressor = Compressor()
    compressor.compress_and_save(
        model=model,
        base_path=base_path,
        dim=dim,
    )
    vels = list(map(str, sorted(compressor.parser.get_vels(
        model='data_wage',
        base_path='data',
        dimpath='low_dim',
    ), key=lambda x: str(x))))
    mesh_data = compressor.parser.get_mesh_data(
        path_to_vel=vels[0],
    )
    compressor.save_mesh(mesh_data=mesh_data)
    compressor.copy_not_compressable(
        model=model,
        base_path=base_path,
        dim=dim,
    )

In [ ]:
doall(
    model='data_wage',
    base_path='data',
    dim='low_dim'
)